In [65]:
import pandas as pd


In [66]:
bus_route = pd.read_csv("../data/정류소_노선.csv")
bus_weight = pd.read_csv("../data/usage_id.csv")
bus_frame = pd.read_csv("../data/busstop.csv")

In [67]:
bus_weight = bus_weight.dropna()

In [68]:
bus_route = bus_route.rename(columns={
    "rigeon1" : "citycode", 
    "nodeid1" : "nodeid", 
    "routeid1" : "routeid"})

In [69]:
bus_frame = bus_frame.rename(columns={"정류장번호" : "nodeid"})

In [70]:
bus_weight = bus_weight.rename(columns={"usage" : "weight"}) ## pd.DataFrame({"routeid" :bus_weight["routeid"], "weight" :bus_weight["usage"]})

# routeid1 열을 explode하여 개별 routeid로 분리
bus_route["routeid"] = bus_route["routeid"].apply(lambda x: eval(x))
exploded_bus_route = bus_route.explode('routeid').reset_index(drop=True)

# bus_route와 bus_weight를 routeid1 기준으로 병합
merged_df = pd.merge(exploded_bus_route, bus_weight[["routeid", "weight"]], on="routeid", how="left")

# nodeid1별로 weight를 합산
nodeid_weight = merged_df.groupby('nodeid')['weight'].sum().reset_index()


## 기존 총데이터에 가중치 추가
busstop_weight = pd.merge(bus_frame, nodeid_weight, on='nodeid', how='left') 





# 결과를 bus_route에 병합하여 가중치 업데이트
# 먼저 bus_route를 explode하여 각 nodeid1에 대한 weight를 병합
#bus_route_exploded = bus_route.explode('routeid1').reset_index(drop=True)
#bus_route_with_weight = pd.merge(bus_route_exploded, bus_weight, on='routeid1', how='left')

# 원래 형태로 복구하면서 nodeid1별 weight를 합산
#final_result = bus_route_with_weight.groupby(['rigeon1', 'nodeid1']).agg({
#    'routeid1': lambda x: list(x),
#    'weight': 'sum'
#}).reset_index()


In [72]:
busstop_weight

,nodeid,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명,geometry,weight
0,CWB379000566,구암고등학교,35.246296,128.593978,2023-10-16,420615.0,38010,경상남도 창원시,창원,POINT (1099529.3099473445 1695090.3215359906),478654.0
1,CWB379000567,사격장입구,35.244923,128.683156,2023-10-16,107508.0,38010,경상남도 창원시,창원,POINT (1107644.8457543373 1695031.1020157929),464401.0
2,CWB379000568,장애인복지회관입구,35.244960,128.673597,2023-10-16,119117.0,38010,경상남도 창원시,창원,POINT (1106775.0680812967 1695024.859684883),732161.0
3,CWB379000570,장애인복지회관입구,35.244773,128.673243,2023-10-16,119118.0,38010,경상남도 창원시,창원,POINT (1106743.105090053 1695003.6802969584),266638.0
4,CWB379000573,지귀상가,35.244778,128.659437,2023-10-16,119111.0,38010,경상남도 창원시,창원,POINT (1105486.9448440736 1694989.4555102403),1120637.0
...,...,...,...,...,...,...,...,...,...,...,...
18854,YSB5166,사과나무집,35.491970,128.989600,2023-10-16,5166.0,38100,경상남도 양산시,양산,POINT (1135114.8887175242 1722807.740568875),3171.0
18855,YSB5167,이지더원3차,35.322630,129.027700,2023-10-16,5167.0,38100,경상남도 양산시,양산,POINT (1138861.2639738221 1704077.9474447218),147088.0
18856,YSB5168,반도유보라6차,35.317890,129.027500,2023-10-16,5168.0,38100,경상남도 양산시,양산,POINT (1138851.1887177934 1703551.928831223),147088.0
18857,YSB5169,양우내안애6차,35.318010,129.023400,2023-10-16,5169.0,38100,경상남도 양산시,양산,POINT (1138478.2608111338 1703559.5004212412),147088.0


In [73]:
busstop_weight.loc[busstop_weight["도시코드"] == 38020, "도시코드"] = 38010
busstop_weight.loc[busstop_weight["도시코드"] == 38040, "도시코드"] = 38010

In [74]:
busstop_weight.groupby(by="도시코드").apply(lambda x: x["weight"].mean())

C:\Users\ghckd\AppData\Local\Temp\ipykernel_2240\2019217515.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  busstop_weight.groupby(by="도시코드").apply(lambda x: x["weight"].mean())


도시코드
38010    234263.973777
38030    303978.659116
38050     37224.522368
38060      8277.291372
38070     93934.365396
38080      7996.443896
38090     42124.013245
38100    110567.758594
38310       718.586552
38320      3771.669988
38330      4145.464856
38340      6148.771131
38350      1624.699099
38360      1507.641745
38380      2220.038941
38390      1896.161785
38400       549.546584
dtype: float64

In [78]:
busstop_weight["weight"] = busstop_weight[["도시코드", "weight"]].groupby(by="도시코드").transform(lambda x: x.fillna(x.mean()))

In [80]:
# busstop_weight.to_csv("../data/busstop_weight.csv", index=False)